<center>

#Modelo Integrado de Predicción de Peso y Optimización de Cosecha en la Producción en la industria avícola para la Reducción de Costos y Cumplimiento de mix de oferta#
##Trabajo de grado II##
##ESTE MODELO FUNCIONÓ PARA 3 DÍAS DE COSECHA##
*Luisa F. Giraldo - Juan S. Guzmán*

</center>

In [ ]:
import pandas as pd

# Paso 1: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install -y coinor-cbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-2 [465 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-2 [275 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-3 [420 kB]
Get:5 ht

# **0. LECTURA DATA**

In [ ]:
#@title Lectura Datos pos Granja y Galpon


# Ruta del archivo (ajustala según tu estructura de Drive)
excel_path = "/content/drive/MyDrive/TDG II/INPUT MODELO OPTIMIZACIÓN.xlsx"

# Leer la hoja
df = pd.read_excel(excel_path, sheet_name='Input_Granjas')


# Inicializar diccionarios
W, ADG, CAT, N, TIPO = {}, {}, {}, {}, {}
#W: peso proyectado

#ADG: ganancia diaria de peso
#CAT: categoría de peso
#N: número de aves en el grupo
#TIPO: tipo de pollo (Blanco o Campesino)

# Recorrer cada fila
for _, row in df.iterrows():
    g = row['GRANJA']
    p = row['GALPON']
    s = row['SEXO']

    N[(g, p, s)] = row['SALDO']
    TIPO[(g, p, s)] = row['TIPO']

    for d in range(1, 7):
        if d == 1:
            col_weight = 'WEIGTH'
            col_adg = 'ADG'
            col_cat = 'CAT PESO'
        else:
            col_weight = f'WEIGTH{d}'
            col_adg = f'ADG{d}'
            col_cat = f'CAT PESO{d}'

        try:
            if pd.notnull(row[col_weight]):
                W[(g, p, s, d)] = float(row[col_weight])
            if pd.notnull(row[col_adg]):
                ADG[(g, p, s, d)] = float(row[col_adg])
            if pd.notnull(row[col_cat]):
                CAT[(g, p, s, d)] = str(row[col_cat]).strip().upper()
        except KeyError as e:
            print(f"⚠️ Columna no encontrada para día {d}: {e}")
        except ValueError as e:
            print(f"❗ Valor no numérico en {g}, {p}, día {d}: {e}")

In [ ]:
#@title Validación Lectura Datos pos Granja y Galpon
# Mostrar cuántos elementos tiene cada parámetro
print(f"🔢 Total entradas en W: {len(W)}")
print(f"🔢 Total entradas en ADG: {len(ADG)}")
print(f"🔢 Total entradas en CAT: {len(CAT)}")
print(f"🔢 Total entradas en N: {len(N)}")
print(f"🔢 Total entradas en TIPO: {len(TIPO)}")

# Mostrar los primeros 5 valores de cada uno
print("\n📦 Ejemplos de W:")
for k in list(W)[:5]:
    print(f"{k}: {W[k]}")

print("\n📦 Ejemplos de ADG:")
for k in list(ADG)[:5]:
    print(f"{k}: {ADG[k]}")

print("\n📦 Ejemplos de CAT:")
for k in list(CAT)[:5]:
    print(f"{k}: {CAT[k]}")

print("\n📦 Ejemplos de N:")
for k in list(N)[:5]:
    print(f"{k}: {N[k]}")

print("\n📦 Ejemplos de TIPO:")
for k in list(TIPO)[:5]:
    print(f"{k}: {TIPO[k]}")


🔢 Total entradas en W: 888
🔢 Total entradas en ADG: 888
🔢 Total entradas en CAT: 888
🔢 Total entradas en N: 148
🔢 Total entradas en TIPO: 148

📦 Ejemplos de W:
('SAN MIGUEL', 5, 'M', 1): 3.277513000000006
('SAN MIGUEL', 5, 'M', 2): 3.328340000000008
('SAN MIGUEL', 5, 'M', 3): 3.375599000000001
('SAN MIGUEL', 5, 'M', 4): 3.420478000000003
('SAN MIGUEL', 5, 'M', 5): 3.4641649999999977

📦 Ejemplos de ADG:
('SAN MIGUEL', 5, 'M', 1): 72.83362222222235
('SAN MIGUEL', 5, 'M', 2): 72.35521739130452
('SAN MIGUEL', 5, 'M', 3): 71.82125531914896
('SAN MIGUEL', 5, 'M', 4): 71.25995833333339
('SAN MIGUEL', 5, 'M', 5): 70.69724489795914

📦 Ejemplos de CAT:
('SAN MIGUEL', 5, 'M', 1): GRANDE
('SAN MIGUEL', 5, 'M', 2): GRANDE
('SAN MIGUEL', 5, 'M', 3): GRANDE
('SAN MIGUEL', 5, 'M', 4): GRANDE
('SAN MIGUEL', 5, 'M', 5): GRANDE

📦 Ejemplos de N:
('SAN MIGUEL', 5, 'M'): 8550
('MI GRANJITA', 1, 'M'): 8000
('MI GRANJITA', 2, 'M'): 16850
('MI GRANJITA', 4, 'M'): 8000
('MI GRANJITA', 6, 'M'): 9500

📦 Ejemplos

In [ ]:
#@title Lectura Datos Demanda por día

# Leer hoja Input_Demanda sin encabezado
demanda_df = pd.read_excel(excel_path, sheet_name='Input_Demanda', header=4)

# Filtrar filas con valores válidos de tipo y categoría
valores_validos = ['CAMPESINO', 'BLANCO']
categorias_validas = ['PEQUEÑO', 'MEDIANO', 'GRANDE']

demanda_df = demanda_df[
    demanda_df['Tipo'].isin(valores_validos) &
    demanda_df['Categoria Peso'].isin(categorias_validas)
]

# Leer nuevamente la hoja completa sin encabezado
demanda_raw = pd.read_excel(excel_path, sheet_name='Input_Demanda', header=None)

# Extraer la tolerancia (celda fila 1, columna 2)
tol = float(demanda_raw.iloc[1, 2])
print(f"Tolerancia (tol): {tol}")


Q = {}
for _, row in demanda_df.iterrows():
    tipo = row['Tipo']
    categoria = row['Categoria Peso']
    for d in range(1, 22):  # porque hay columnas hasta Day21
        valor = row.get(f'Day{d}', None)
        if pd.notnull(valor):
            Q[(tipo, categoria, d)] = valor

tipos = sorted(set(k[0] for k in Q))
categorias = sorted(set(k[1] for k in Q))

print("Tipos únicos encontrados:", tipos)
print("Categorías únicas encontradas:", categorias)


Tolerancia (tol): 0.1
Tipos únicos encontrados: ['BLANCO', 'CAMPESINO']
Categorías únicas encontradas: ['GRANDE', 'MEDIANO', 'PEQUEÑO']


Validamos la lectura de la demanda por día

In [ ]:
tipos = sorted(set([k[0] for k in Q.keys()]))
categorias = sorted(set([k[1] for k in Q.keys()]))

print("Tipos únicos encontrados:", tipos)
print("Categorías únicas encontradas:", categorias)

print("\nDemandas por tipo y categoría:")
for tipo in tipos:
    for cat in categorias:
        dias = [Q.get((tipo, cat, d), '-') for d in range(1, 7)]
        print(f"{tipo} / {cat}: {dias}")


Tipos únicos encontrados: ['BLANCO', 'CAMPESINO']
Categorías únicas encontradas: ['GRANDE', 'MEDIANO', 'PEQUEÑO']

Demandas por tipo y categoría:
BLANCO / GRANDE: [15000.0, 15000, 15000, 15000.0, 15000.0, 15000.0]
BLANCO / MEDIANO: [67815.0, 67815, 67815, 67815.0, 67815.0, 67815.0]
BLANCO / PEQUEÑO: [24811.0, 24811, 24811, 24811.0, 24811.0, 24811.0]
CAMPESINO / GRANDE: [58068.0, 58068, 58068, 58068.0, 58068.0, 58068.0]
CAMPESINO / MEDIANO: [0.0, 0, 0, 0.0, 0.0, 0.0]
CAMPESINO / PEQUEÑO: [0.0, 0, 0, 0.0, 0.0, 0.0]


In [ ]:
# ---------------------------
# Leer parámetros semanales
# ---------------------------
# Leer nuevamente la hoja completa sin encabezado
demanda_raw = pd.read_excel(excel_path, sheet_name='Input_Demanda', header=None)

# 🔹 Leer TOT (Total de aves por semana)
TOT = {}
fila_tot = 15  # Fila 16 en Excel → index 15
col_tot = 5    # Columna E en Excel → index 4
valor = demanda_raw.iloc[fila_tot, col_tot]
if pd.notnull(valor):
    TOT[1] = float(valor)  # Solo se ha definido para semana 1 por ahora

print("\nTOT (total de aves deseadas por semana):")
print(TOT)

# 🔹 Leer MAPD (día → semana)
MAPD = {}
for i in range(13, 34):  # Filas 14 a 34 en Excel
    semana = demanda_raw.iloc[i, 1]
    dia = demanda_raw.iloc[i, 2]
    if pd.notnull(semana) and pd.notnull(dia):
        MAPD[int(dia)] = int(semana)

print("\nMAPD (día → semana):")
print(MAPD)



TOT (total de aves deseadas por semana):
{1: 994164.0}

MAPD (día → semana):
{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2, 13: 2, 14: 2, 15: 3, 16: 3, 17: 3, 18: 3, 19: 3, 20: 3, 21: 3}


# **1. Modelo de Optimización**

In [ ]:
#@title Inicialización del modelo y definición de conjuntos

from pyomo.environ import *

model = ConcreteModel()

# Conjuntos base
model.GP = Set(initialize=set((g, p, s) for (g, p, s) in N.keys()))  # (Granja, Galpón, Sexo)
model.D = RangeSet(1, 6)  # Días del horizonte
model.T = Set(initialize=set(t for t, _, _ in Q.keys()))  # Tipos de pollo
model.C = Set(initialize=set(c for _, c, _ in Q.keys()))  # Categorías de peso


In [ ]:
#@title Parámetros: Demanda, Tolerancia, ADG, Peso, Categoría, Saldo y Tipo

# Filtrar los diccionarios antes de definir los parámetros

# Solo incluir demanda para días presentes en model.D
Q_filtrado = {(t, c, d): v for (t, c, d), v in Q.items() if d in model.D}


# Solo incluir pesos, adg y categorías para días válidos
W_filtrado = {(g, p, s, d): v for (g, p, s, d), v in W.items() if d in model.D}
ADG_filtrado = {(g, p, s, d): v for (g, p, s, d), v in ADG.items() if d in model.D}
CAT_filtrado = {(g, p, s, d): v for (g, p, s, d), v in CAT.items() if d in model.D}
MAPD_filtrado = {d: w for d, w in MAPD.items() if d in model.D}

# Conjunto de semanas
model.WEEK = Set(initialize=set(TOT.keys()))

# Total deseado de aves beneficiadas por semana
model.TOT = Param(model.WEEK, initialize=TOT, default=0)

# Mapeo de día → semana
model.MAPD = Param(model.D, initialize=MAPD_filtrado, default=0)

# Parámetros en Pyomo con datos filtrados

# Demanda deseada
model.Q = Param(model.T, model.C, model.D, initialize=Q_filtrado, default=0)

# Tolerancia de demanda
model.tol = Param(initialize=tol)

# Peso proyectado
model.W = Param(model.GP, model.D, initialize=W_filtrado, default=0)

# Ganancia diaria de peso (ADG)
model.ADG = Param(model.GP, model.D, initialize=ADG_filtrado, default=0)

# Categoría de peso (Pequeño, Mediano, Grande)
model.CAT = Param(model.GP, model.D, initialize=CAT_filtrado, default='-')

# Saldo de aves
model.N = Param(model.GP, initialize=N)

# Tipo de pollo (Blanco o Campesino)
model.Tipo = Param(model.GP, initialize=TIPO)


default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.11/dist-packages/pyomo/core/base/block.py:1101)
default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', you can suppress this warning by
explicitly specifying 'within=Any' to the Param constructor.
(deprecated in 5.6.9, will be removed in (or after) 6.0)
(called from /usr/local/lib/python3.11/dist-packages/pyomo/core/base/indexed_component.py:718)
The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend
the domain of this Paramto be 'Any', yo

In [ ]:
#@title Variables de decisión y auxiliares

from pyomo.environ import *

# Variable principal: cosecha de grupos
# x[g,p,s,d] = 1 si se cosecha el grupo (g,p,s) el día d; 0 en caso contrario.
model.x = Var(model.GP, model.D, domain=Binary)

#  Variable auxiliar: desviación absoluta entre lo cosechado y lo deseado por tipo, categoría y día
# dev[t,c,d] ≥ |(pollos cosechados) - (cantidad deseada Q)|
model.dev = Var(model.T, model.C, model.D, domain=NonNegativeReals)


In [ ]:
#@title Restricción R1: Cada grupo (g, p, s) puede cosecharse como máximo una vez en el horizonte:

def cosecha_una_vez_rule(m, g, p, s):
    return sum(m.x[(g, p, s), d] for d in m.D) <= 1

model.r1_cosecha_una_vez = Constraint(model.GP, rule=cosecha_una_vez_rule)

In [ ]:
#@title Restricción R2:  Cumplimiento del mix deseado con tolerancia

#Incorporamos la variable auxiliar dev para medir la desviación absoluta respecto al valor objetivo por tipo, categoría y día:

# Variable auxiliar para la desviación absoluta
model.dev = Var(model.T, model.C, model.D, domain=NonNegativeReals)

# Restricciones para modelar |cantidad_real - cantidad_deseada| ≤ dev
def desviacion_superior_rule(m, t, c, d):
    cantidad_real = sum(m.N[gps] * m.x[gps, d]
                        for gps in m.GP
                        if m.Tipo[gps] == t and m.CAT[gps, d] == c)
    return cantidad_real - m.Q[t, c, d] <= m.dev[t, c, d]

def desviacion_inferior_rule(m, t, c, d):
    cantidad_real = sum(m.N[gps] * m.x[gps, d]
                        for gps in m.GP
                        if m.Tipo[gps] == t and m.CAT[gps, d] == c)
    return m.Q[t, c, d] - cantidad_real <= m.dev[t, c, d]

model.r2_dev_superior = Constraint(model.T, model.C, model.D, rule=desviacion_superior_rule)
model.r2_dev_inferior = Constraint(model.T, model.C, model.D, rule=desviacion_inferior_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
#@title Restricción R3: Prioridad de hembras sobre machos

#Si en un mismo galpón hay tanto hembras como machos, los machos solo pueden cosecharse mínimo 4 días después de que se hayan cosechado las hembras.

#Supuestos para implementar R3:
#Esta restricción aplica solo a galpones compartidos, es decir, donde existan ambos sexos: 'Hembra' y 'Macho'.
#Se necesita revisar si el grupo (g, p, 'Macho') está siendo cosechado el día d, y si el grupo (g, p, 'Hembra') fue cosechado antes.

# 📌 Restricción R3: los machos solo pueden cosecharse al menos 4 días después que las hembras del mismo galpón

def prioridad_hembras_rule(m, g, p, d):
    if d < 5:
        return Constraint.Skip  # No aplica para los primeros 4 días

    return m.x[(g, p, 'MACHO'), d] <= 1 - sum(
        m.x[(g, p, 'HEMBRA'), d_] for d_ in range(d - 4, d)
        if (g, p, 'HEMBRA') in m.GP and d_ in m.D
    )

model.r3_prioridad_hembras = ConstraintList()
for g, p, s in model.GP:
    if s == 'MACHO':
        for d in model.D:
            model.r3_prioridad_hembras.add(prioridad_hembras_rule(model, g, p, d))

In [ ]:
#@title Restricción R5: Total de aves deseadas por semana

#model.MAPD[d] para agrupar los días por semana:

def total_semanal_rule(m, w):
    dias_semana = [d for d in m.D if m.MAPD[d] == w]
    return inequality(
        (1 - m.tol) * m.TOT[w],
        sum(m.N[gps] * m.x[gps, d] for gps in m.GP for d in dias_semana),
        (1 + m.tol) * m.TOT[w]
    )

model.r5_total_semanal = Constraint(model.WEEK, rule=total_semanal_rule)


In [ ]:
tol

0.1

In [ ]:
# Variable auxiliar: desviación absoluta entre cosecha y demanda
model.dev = Var(model.T, model.C, model.D, domain=NonNegativeReals)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
# Restricciones: modelar desviación absoluta para cada tipo, categoría y día
def desv_pos_rule(m, t, c, d):
    cosechado = sum(m.N[g, p, s] * m.x[g, p, s, d]
                    for (g, p, s) in m.GP
                    if m.Tipo[g, p, s] == t and m.CAT[g, p, s, d] == c)
    return cosechado - m.Q[t, c, d] <= m.dev[t, c, d]

def desv_neg_rule(m, t, c, d):
    cosechado = sum(m.N[g, p, s] * m.x[g, p, s, d]
                    for (g, p, s) in m.GP
                    if m.Tipo[g, p, s] == t and m.CAT[g, p, s, d] == c)
    return m.Q[t, c, d] - cosechado <= m.dev[t, c, d]

model.DesvPositiva = Constraint(model.T, model.C, model.D, rule=desv_pos_rule)
model.DesvNegativa = Constraint(model.T, model.C, model.D, rule=desv_neg_rule)


model.x → Toma decisiones de en qué día cosechar cada grupo.

model.dev → Mide la desviación diaria entre lo cosechado y lo que se pidió por tipo y categoría. Esta variable es usada en la función objetivo para minimizar los desvíos respecto al mix deseado.

In [ ]:
#@title Función objetivo: minimizar la desviación del mix diario

def obj_rule(m):
    return sum(m.dev[t, c, d] for t in m.T for c in m.C for d in m.D)

model.obj = Objective(rule=obj_rule, sense=minimize)

In [ ]:
!which cbc


/usr/bin/cbc


In [ ]:
#@title Optimización maximizar la ganancia diaria total (ADG)
# 📌 Función objetivo: minimizar desviación absoluta respecto al mix
def obj_rule(m):
    return sum(m.dev[t, c, d] for t in m.T for c in m.C for d in m.D)

model.obj = Objective(rule=obj_rule, sense=minimize)

# 📌 Resolver con CBC usando límite de tiempo
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
import logging
from pyomo.util.infeasible import log_infeasible_constraints

solver = SolverFactory('cbc', executable='/usr/bin/cbc')
solver.options["logLevel"] = "2"

tiempo_maximo = 10800  # 15 minutos

print(f"⏳ Ejecutando solver con límite de {tiempo_maximo} segundos...\n")
result = solver.solve(model, tee=True, options={'seconds': tiempo_maximo})

# 👇 Habilitar impresión de restricciones infactibles
logging.getLogger('pyomo.util.infeasible').setLevel(logging.INFO)
log_infeasible_constraints(model)

ruta_excel = "/content/drive/MyDrive/TDG II/resultado_cosechas.xlsx"

# 📊 Mostrar resumen de cosechas encontradas (si hay)
if result.solver.termination_condition in [TerminationCondition.optimal, TerminationCondition.feasible] and \
   result.solver.status in [SolverStatus.ok, SolverStatus.aborted, SolverStatus.warning]:

    print("\n📦 Cosechas programadas:")
    resultados = []

    for g, p, s, d in model.x:
        if model.x[g, p, s, d].value == 1:
            print(f"Grupo {g} - Galpón {p} - Sexo {s} se cosecha el día {d}")
            resultados.append({
                'GRANJA': g,
                'GALPON': p,
                'SEXO': s,
                'DIA_COSECHA': d,
                'TIPO': model.Tipo[g, p, s],
                'SALDO': model.N[g, p, s],
                'CATEGORIA_PESO': model.CAT[g, p, s, d],
                'ADG': model.ADG[g, p, s, d],
                'PESO': model.W[g, p, s, d]
            })

    # Exportar resultados a Excel
    df_resultado = pd.DataFrame(resultados)

    df_resultado.to_excel(ruta_excel, index=False)
    print(f"✅ Resultados exportados correctamente a: {ruta_excel}")

else:
    print("⚠️ El modelo es infactible o no encontró una solución viable dentro del tiempo dado.")


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Node 5294551 depth 272 unsatisfied 9 sum 1.14633e+13 obj 414768 guess 414769 branching on -1
Node 5294552 depth 273 unsatisfied 6 sum 7.77506e+12 obj 414768 guess 414769 branching on -1
Node 5294553 depth 274 unsatisfied 7 sum 7.66915e+12 obj 414768 guess 414769 branching on -1
Node 5294554 depth 275 unsatisfied 6 sum 5.98347e+12 obj 414768 guess 414769 branching on -1
Node 5294555 depth 275 unsatisfied 3 sum 4.9705e+12 obj 414768 guess 414769 branching on -1
Node 5294556 depth 274 unsatisfied 3 sum 4.9705e+12 obj 414768 guess 414769 branching on -1
Node 5294557 depth 273 unsatisfied 6 sum 7.56911e+12 obj 414768 guess 414769 branching on -1
Node 5294558 depth 274 unsatisfied 6 sum 7.78217e+12 obj 414768 guess 414769 branching on -1
Node 5294559 depth 275 unsatisfied 6 sum 7.10461e+12 obj 414768 guess 414769 branching on -1
Node 5294560 depth 276 unsatisfied 9 sum 1.35663e+13 obj 414768 guess 414770 branching on -1
Node 

⚠️ El modelo es infactible o no encontró una solución viable dentro del tiempo dado.


In [ ]:
#@title Exportar Resultados

import os

if hasattr(model, 'dev'):
    print("\n📉 Desviaciones en el cumplimiento del mix diario (por tipo y categoría):")
    resumen_desv = []

    for (t, c, d) in model.dev:
        valor_dev = model.dev[t, c, d].value
        if valor_dev is not None and valor_dev > 0:
            print(f"Tipo: {t} | Categoría: {c} | Día: {d} → Desviación: {valor_dev:.2f}")
            resumen_desv.append({
                'TIPO': t,
                'CATEGORIA': c,
                'DIA': d,
                'DESVIACION': valor_dev
            })

    if resumen_desv:
        df_desv = pd.DataFrame(resumen_desv)

        if os.path.exists(ruta_excel):
            # Añadir hoja si el archivo ya existe
            with pd.ExcelWriter(ruta_excel, engine='openpyxl', mode='a') as writer:
                df_desv.to_excel(writer, sheet_name='Desviaciones', index=False)
            print(f"📄 Desviaciones exportadas a hoja 'Desviaciones' en el mismo archivo Excel.")
        else:
            # Crear nuevo archivo con una sola hoja
            with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
                df_desv.to_excel(writer, sheet_name='Desviaciones', index=False)
            print(f"📄 Archivo Excel creado con hoja 'Desviaciones'.")
    else:
        print("✅ No se encontraron desviaciones. La demanda fue completamente cumplida.")



📉 Desviaciones en el cumplimiento del mix diario (por tipo y categoría):
Tipo: CAMPESINO | Categoría: GRANDE | Día: 1 → Desviación: 153.00
Tipo: CAMPESINO | Categoría: GRANDE | Día: 2 → Desviación: 1943.00
Tipo: CAMPESINO | Categoría: GRANDE | Día: 3 → Desviación: 3865.00
Tipo: CAMPESINO | Categoría: GRANDE | Día: 4 → Desviación: 3182.00
Tipo: CAMPESINO | Categoría: GRANDE | Día: 5 → Desviación: 392.00
Tipo: CAMPESINO | Categoría: GRANDE | Día: 6 → Desviación: 11550.00
Tipo: CAMPESINO | Categoría: MEDIANO | Día: 2 → Desviación: 15980.00
Tipo: CAMPESINO | Categoría: MEDIANO | Día: 5 → Desviación: 15040.00
Tipo: CAMPESINO | Categoría: PEQUEÑO | Día: 3 → Desviación: 17390.00
Tipo: CAMPESINO | Categoría: PEQUEÑO | Día: 5 → Desviación: 28576.00
Tipo: BLANCO | Categoría: GRANDE | Día: 1 → Desviación: 1580.00
Tipo: BLANCO | Categoría: GRANDE | Día: 2 → Desviación: 6270.00
Tipo: BLANCO | Categoría: GRANDE | Día: 3 → Desviación: 15000.00
Tipo: BLANCO | Categoría: GRANDE | Día: 4 → Desviación: 

In [ ]:
resultados_detalle = []

for (g, p, s, d) in model.x:
    if model.x[g, p, s, d].value == 1:
        resultados_detalle.append({
            'DIA': d,
            'GRANJA': g,
            'GALPON': p,
            'SEXO': s,
            'TIPO': model.Tipo[g, p, s],
            'CATEGORIA': model.CAT[g, p, s, d],
            'PESO': model.W[g, p, s, d],
            'ADG': model.ADG[g, p, s, d],
            'CANTIDAD_AVES': model.N[g, p, s]
        })

# Convertir a DataFrame
df_cosechas = pd.DataFrame(resultados_detalle)

# Exportar a Excel (nueva hoja o nuevo archivo)
if not df_cosechas.empty:
    if os.path.exists(ruta_excel):
        with pd.ExcelWriter(ruta_excel, engine='openpyxl', mode='a') as writer:
            df_cosechas.to_excel(writer, sheet_name='Detalle_Cosechas', index=False)
        print("📄 Detalle de cosechas exportado a hoja 'Detalle_Cosechas'.")
    else:
        with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
            df_cosechas.to_excel(writer, sheet_name='Detalle_Cosechas', index=False)
        print("📄 Archivo Excel creado con hoja 'Detalle_Cosechas'.")
else:
    print("⚠️ No se encontraron grupos cosechados para exportar.")


📄 Detalle de cosechas exportado a hoja 'Detalle_Cosechas'.
